# How the MAGI 2 database was built

This notebook describes how the MAGI 2 database was built, in case you want to change something, or if an update is needed. We will describe two parts, namely how the Retro Rules database was altered to be suitable for MAGI 2 and how we obtained protein sequence information.

**Author: Hanneke Leegwater**  
**Last edited on: 28 July 2020**

* [0. Load python packages](#header_0)
* [1. Build the database framework](#header_1)
* [2. Preprocessing the Retro Rules database](#header_2)
* [3. Precomputing reactions for all substrates](#header_3)
* [4. Downloading protein sequences](#header_4)
* [5. Make refseq pandas dataframe for reaction_to_gene search](#header_5)

## 0. Load python packages <a class="anchor" id="header_0"></a>
Note that not all parts were performed in Python. Also, set your path to the MAGI database here.

In [ ]:
import os
import sys
import pandas as pd
import sqlite3
import re
## you might need this if rdkit does not work
os.environ["PATH"] += r";C:\Users\hanne\Anaconda3\envs\magi_2\Library\bin"
import rdkit
magi_path = "/Users/hanne/Documents/GitHub/magi"
if os.path.exists(magi_path):
    sys.path.insert(0,magi_path)
    from workflow_2 import compound_to_reaction as magi
    from workflow_2 import gene_to_reaction as magi_g2r
else: 
    print("Path to MAGI does not exist")

## 1. Build the database framework <a class="anchor" id="header_"></a>
Use the MAGI_database.sql file to build a database in your preferred database program and set the path here.

In [ ]:
#database_path = os.path.join(magi_path, "workflow_2", "database", "MAGI_database.db")
database_path = "./MAGI_database.db"

In [ ]:
connection = sqlite3.connect(database_path)

## 2. Preprocessing the Retro Rules database <a class="anchor" id="header_2"></a>
- We downloaded the Retro Rules preformatted database (rr02 with implicit hydrogens for RetroPath RL) from retrorules.org. We chose to use this file and not their database itself, because the preformatted tables also contain substrate SMILES, which we need for MAGI 2. The file we use is called retrorules_rr02_rp3_nohs/retrorules_rr02_flat_all.tsv. 
- We removed reactions with just H2O as a substrate and removed columns that MAGI 2 does not need. 
- Next, we calculated canonical SMILES for all substrates, which will be used to look up precomputed reactions. This took about 4 hours on a single CPU. Canonical SMILES were added to the Retro Rules dataframe in a column called Canonical_SMILES. 
- InChI keys were calculated for all canonical SMILES to speed up the lookup of compounds. These were stored in a column called InchiKey. 
- Substrates and reactions were stored in the MAGI database.

In [ ]:
retro_rules_path = "./retrorules_rr02_rp3_nohs/retrorules_rr02_flat_all.tsv"
retro_rules = pd.read_csv(retro_rules_path, sep = "\t")
water_smiles = "[OH2]"
retro_rules = retro_rules[retro_rules["Substrate_SMILES"] != water_smiles]
retro_rules = retro_rules[["Reaction_ID", "Diameter", "Rule_SMARTS", "Substrate_ID", "Substrate_SMILES", "Rule_SMILES"]]

In [ ]:
substrate_smiles_all = list(set(retro_rules["Substrate_SMILES"]))
print("Total nr of SMILES to process is: {}".format(len(substrate_smiles_all)))
substrate_smiles_dict = {}
for smiles in substrate_smiles_all:
    canonical_smiles = magi.prepare_smiles(smiles)
    substrate_smiles_dict[smiles] = canonical_smiles

In [ ]:
def substrate_lookup(smiles):
    return substrate_smiles_dict[smiles]
retro_rules["Canonical_SMILES"] = retro_rules["Substrate_SMILES"].apply(substrate_lookup)

substrate_objects_lookup = {}
for smiles in list(set(retro_rules["Canonical_SMILES"])):
    substrate_objects_lookup[smiles] = magi.mol_from_smiles(smiles, useHs = False)
def magi_mol_from_smiles_lookup(smiles):
    return substrate_objects_lookup[smiles]
retro_rules["InchiKey"] = retro_rules["Substrate"].apply(inchi.MolToInchiKey)

In [ ]:
retro_rules_substrates = retro_rules[["Substrate_ID", "Substrate_SMILES", "Canonical_SMILES", "InchiKey"]].copy()
retro_rules_substrates.columns = ["retro_rules_ID", "retro_rules_smiles","canonical_smiles","inchi_key"]
retro_rules_substrates.drop_duplicates(inplace = True)
retro_rules_substrates.to_sql('Retro_rules_substrates', con = connection, if_exists = "replace", chunksize = 1000, index_label = "substrate_ID")
connection.commit()

In [ ]:
subs = {}
for index, row in retro_rules_substrates.iterrows():
    subs[row["Substrate_ID"]] = index
def substitute_substrate_ID(ID):
    return subs[ID]
retro_rules_reactions = retro_rules[["Reaction_ID", "Rule_SMARTS", "Substrate_ID", "Diameter"]].copy()
retro_rules_reactions.drop_duplicates(inplace = True)
retro_rules_reactions.reset_index(drop = True, inplace = True)
retro_rules_reactions.columns = ["retro_rules_ID", "retro_rules_smarts","substrate_ID","diameter"]
retro_rules_reactions["substrate_ID"] = retro_rules_reactions["substrate_ID"].apply(substitute_substrate_ID)
retro_rules_reactions.to_sql('Retro_rules_reactions', con = connection, if_exists = "replace", chunksize = 1000, index_label = "reaction_ID")
connection.commit()

## 3. Precomputing reactions for all substrates <a class="anchor" id="header_3"></a>
This part was performed by using the The National Energy Research Scientific Computing Center facility at LBL. We subsetted all substrates from the Retro Rules table and calculated all reactions with minimum similarity 0.6 and minimum diameter 2 so that a molecule can be looked up if MAGI has calculated reactions before. We subsetted all substrates in 25 files. All scripts and input files for these MAGI runs can be found in the folder 20200131_MAGI_precomputing_reactions. However, if you update the Retro Rules reaction patterns, you will probably have to re-run the compound to reaction searches with updated substrate lists.

In [ ]:
path_to_precomputed_c2r_files = "./MAGI_precomputing_reactions/"

In [ ]:
precomputed_c2r_table_total = pd.DataFrame(columns = ["original_compound","reaction_ID","similarity"])
for nr in range(1,26):
    filename = "output_{}/intermediate_files/compound_to_reaction.csv".format(str(nr))
    print("Opening {} ...".format(filename), end = "\t")
    precomputed_c2r_table = pd.read_csv(os.path.join(path_to_precomputed_c2r_files, filename))
    precomputed_c2r_table.drop("compound_score", axis = 1, inplace = True)
    precomputed_c2r_table_total = pd.concat([precomputed_c2r_table_total, precomputed_c2r_table])
precomputed_c2r_table_total = precomputed_c2r_table_total.drop_duplicates()
precomputed_c2r_table_total.reset_index(drop = True, inplace = True)

In [ ]:
precomputed_molecules = retro_rules_substrates.copy()
precomputed_molecules["substrate_ID"] = precomputed_molecules.index
precomputed_molecules = precomputed_molecules[["substrate_ID", "retro_rules_smiles","canonical_smiles","inchi_key"]] # reorder
precomputed_molecules.columns = ['molecule_ID', 'smiles', 'canonical_smiles', 'inchi_key']
precomputed_molecules.to_sql('Precomputed_molecules', con = connection, if_exists = 'append', chunksize = 1000, index=False)

In [ ]:
precomputed_c2r_table_total = precomputed_c2r_table_total.merge(precomputed_molecules[["molecule_ID", "smiles"]], how = "left", left_on = "original_compound", right_on = "smiles")
precomputed_c2r_table_total.drop(["original_compound", "smiles"], axis = 1, inplace = True)
precomputed_c2r_table_total.to_sql('Precomputed_reactions', con = connection, if_exists = 'append', chunksize = 1000, index=False)

## 4. Downloading protein sequences <a class="anchor" id="header_4"></a>

#### Getting relevant Rhea identifiers

MAGI 2's protein database is based on Rhea identifiers. First, we obtained all Rhea identifiers for which we needed sequence information from the MAGI 2 database.

In [ ]:
connection = sqlite3.connect(os.path.join(magi_path, "workflow_2", "database", "MAGI_database.db"))
query = "SELECT rhea_ID FROM Retro_rules_to_rhea_reactions"
rhea_IDs = pd.read_sql_query(query, connection)
print(rhea_IDs.shape)
print(rhea_IDs.head())
rhea_IDs.to_csv("rhea_ids.csv", index = False)

#### Downloading a Rhea to UniProt table
Rhea lists UniProt protein identifiers for all Rhea IDs. This table needs to be downloaded manually from https://www.rhea-db.org/download . The table is called rhea2uniprot.tsv and the direct link is ftp://ftp.expasy.org/databases/rhea/tsv/rhea2uniprot.tsv . We subsetted the table to only keep UniProt identifiers for reactions that are in the MAGI 2 database. 

In [ ]:
rhea2uniprot = pd.read_csv("rhea2uniprot.tsv", sep="\t",dtype={"RHEA_ID":str})
print("Full Rhea 2 uniprot table size is {}".format(rhea2uniprot.shape[0]))
print(rhea2uniprot.head())

#### Getting relevant UniProt identifiers

In [ ]:
rhea2uniprot = rhea2uniprot.merge(rhea_IDs, how="right", left_on="RHEA_ID", right_on="rhea_ID")
rhea2uniprot = rhea2uniprot[pd.notna(rhea2uniprot["RHEA_ID"])][["rhea_ID", "ID"]]
rhea2uniprot.columns = ["rhea_ID", "uniprot_ID"]
print("Subset Rhea 2 uniprot table size is {}".format(rhea2uniprot.shape[0]))
print(rhea2uniprot.head())

#### Parsing this to make a list of unique uniprot IDs and write these to a file.

In [ ]:
with open('./sequences.txt','w') as fid:
    fid.write('\n'.join(rhea2uniprot["uniprot_ID"].unique()))

#### Getting UniProt sequences
For this, go to https://www.uniprot.org/uploadlists/ and select the file of uniprot IDs. Select to search to UniProtKB identifier. Download the fasta (canonical) file from the link after it completes.  This takes about 5 minutes. Store this file as rhea2uniprot.fasta in the MAGI database folder. Run the makeblastdb script from NCBI to turn this fasta file into a database. 

#### Add UniProt headers to the MAGI database

In [ ]:
uniprot_headers = {}
with open("./rhea2uniprot.fasta", "r") as fastafile:
    for line in fastafile:
        if line.startswith(">"):
            ## Get ID without > symbol
            identifier = line.split(" ")[0][1:] 
            ## Remove first and third part from identifier
            ## Example sp|A0PSD4|AHPD_MYCUA becomes A0PSD4
            identifier = identifier.split("|")[1] 
            uniprot_headers[identifier] = line.rstrip()[1:]

In [ ]:
uniprot_headers = pd.DataFrame.from_dict(uniprot_headers, orient="index", columns=["header"])

In [ ]:
rhea2uniprot = rhea2uniprot.merge(uniprot_headers, how = "left", left_on = "uniprot_ID", right_index = True)
def get_extended_protein_id(header):
    protein_id = header.split(" ")[0]
    return protein_id
rhea2uniprot["protein_ID"] = rhea2uniprot["header"].apply(get_extended_protein_id)
rhea_to_uniprot.to_sql( 'Proteins', con = connection, if_exists = 'replace', chunksize = 1000, index = False)
connection.commit()

In [ ]:
connection.close()

#### Build retro rules to rhea reactions table
This is a table that links metanetx identifiers to Rhea identifiers. It was downloaded from https://www.metanetx.org/mnxdoc/mnxref.html.

In [ ]:
reactions_info = []
with open("./reac_xref.tsv", "r") as metanetx_file:
    for line in metanetx_file:
        if line.startswith("rhea"):
            reactions_info.append(line)

with open("./reac_xref_rhea.tsv", "w") as metanetx_file:
    for line in reactions_info:
        metanetx_file.write(line)    

In [ ]:
reactions_info = pd.read_csv("./reac_xref_rhea.tsv", sep="\t", header = None)
reactions_info.columns = ["rhea_ID", "retro_rules_ID", "other"]
reactions_info = reactions_info[["retro_rules_ID","rhea_ID"]]
reactions_info["rhea_ID"] = reactions_info["rhea_ID"].str.replace("rheaR:","")
reactions_info = reactions_info[reactions_info['retro_rules_ID'] != "EMPTY"]
reactions_info.to_sql(    'Retro_rules_to_rhea_reactions', con = connection, if_exists = 'replace', chunksize = 1000, index = False)
connection.commit()

## Save database

In [ ]:
connection.commit()

## 5. Make refseq pandas dataframe for reaction_to_gene search <a class="anchor" id="header_5"></a>

In [ ]:
rhea_to_uniprot = pd.read_csv("rhea2uniprot.tsv", sep="\t")
rhea_to_uniprot = rhea_to_uniprot[["RHEA_ID", "ID"]]
rhea_to_uniprot.rename({"RHEA_ID": "rhea_ID", "ID": "uniprot_ID"}, axis = 1,inplace = True)

In [ ]:
uniprot_headers = {}
with open("rhea2uniprot.fasta", "r") as fastafile:
    for line in fastafile:
        if line.startswith(">"):
            ## Get ID without > symbol
            identifier = line.split(" ")[0][1:] 
            ## Remove first and third part from identifier
            ## Example sp|A0PSD4|AHPD_MYCUA becomes A0PSD4
            identifier = identifier.split("|")[1] 
            uniprot_headers[identifier] = line.rstrip()[1:]
uniprot_headers = pd.DataFrame.from_dict(uniprot_headers, orient="index", columns=["header"])

In [ ]:
rhea_to_uniprot = rhea_to_uniprot.merge(uniprot_headers,
                     how = "left",
                     left_on = "uniprot_ID", right_index = True)

#### Check if any headers are missing. This should be 0, otherwise you may miss sequences

In [ ]:
rhea_to_uniprot[pd.isna(rhea_to_uniprot["header"])]

#### This section makes a data frame with gene IDs and gene sequences

In [ ]:
reference_genome = magi_g2r.make_genome_dataframe_from_fasta("./rhea2uniprot.fasta")
def alter_gene_id(gene_id):
    try:
        gene_id = gene_id.split("|")[1] 
    except:
        pass
    return gene_id
reference_genome["Gene_ID"] = reference_genome["Gene_ID"].apply(alter_gene_id)

#### Merge genome sequences to the proteins that we need

In [ ]:
genome = rhea_to_uniprot.merge(reference_genome, how="left", left_on = "uniprot_ID", right_on = "Gene_ID")

#### Remove duplicate gene sequences

In [ ]:
genome = genome[["uniprot_ID", "sequence"]]
genome = genome.drop_duplicates()

#### Write to file to use for MAGI reaction to gene search and place this table in the database folder for MAGI.

In [ ]:
genome.to_csv("./reaction_to_gene_reference.csv", index = False)